In [2]:
import pandas as pd

#sample data load
df = pd.read_csv('./src_data/finalcate.csv', dtype=str)

In [4]:
df.reset_index(inplace = True)

In [5]:
df = df[df['attr_name'] != 'Donor Name :']

In [6]:
df

,index,Unnamed: 0,test_id,test_name,speciment,attr_id,attr_name,unit,result,data_type,noraml_range
0,0,86,10994,Complete Urine Analysis (Urine R/E),URINE,14491.0,Protein,Null,(+),Categorical,NaN
1,1,87,10994,Complete Urine Analysis (Urine R/E),URINE,14493.0,Ketones body,Null,Absent,Categorical,NaN
2,2,90,10994,Complete Urine Analysis (Urine R/E),URINE,14495.0,Urobilinogen,Null,Normal,Categorical,NaN
3,3,91,10994,Complete Urine Analysis (Urine R/E),URINE,14496.0,Pus Cell,Null,0-2,Categorical,NaN
4,4,92,10994,Complete Urine Analysis (Urine R/E),URINE,14497.0,Sediment,Null,Nil,Categorical,NaN
...,...,...,...,...,...,...,...,...,...,...,...
184901,184901,3527682,5030472,Thyroglobulin-(Thyroglobulin),NaN,3002812.0,Thyroglobulin,ng/ml,<0.20,Categorical,Upto 55.0 ng/ml
184902,184902,3527918,101410,Growth Hormone Basal,NaN,103728.0,Growth Hormone Basal,ng/mL,>50,Categorical,0.06 - 5.0 ng/mL
184903,184903,3529196,100151,Anti- Scl 70,NaN,101957.0,Anti- Scl 70,[NEGATIVE],0.58 U/ml,Categorical,<0.95 = Negative\n>0.95 -1.0 = Borderline\n>1....
184904,184904,3529318,5027602,EBV IgM,NaN,3002226.0,EBV IgM,[Negative],<5.0,Categorical,Negative: <09 U/ml Positive: >13 U/ml


In [7]:
df.size

2021261

In [8]:
df = df[['index', 'result']]

In [9]:
df.rename( columns = {
    'index': 'measurement_concept_id',
    'result': 'value_source_value'
}, inplace = True)

/tmp/ipykernel_1267/2740110528.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename( columns = {


In [10]:
df

,measurement_concept_id,value_source_value
0,0,(+)
1,1,Absent
2,2,Normal
3,3,0-2
4,4,Nil
...,...,...
184901,184901,<0.20
184902,184902,>50
184903,184903,0.58 U/ml
184904,184904,<5.0


In [11]:
# 1) Data preparation
from collections import defaultdict
import re

def _general_preprocessing(df):
    
    # window user_encoding
    df['value_source_value'] = df['value_source_value'].str.encode('utf8', errors = 'strict').str.decode('utf8', errors = 'strict')
    
    # lower, strip space, \d\+ restore
    df['value_source_value'] = df['value_source_value'].str.lower()
    df['value_source_value'] = df['value_source_value'].str.strip()
    df['value_source_value'] = df['value_source_value'].str.replace('"',"")
    df['value_source_value'] = df['value_source_value'].str.replace('(',"")
    df['value_source_value'] = df['value_source_value'].str.replace(')',"")
    df['value_source_value'] = df['value_source_value'].str.replace(r'(\d)(\+)', lambda x: x.groups()[1]*int(x.groups()[0]))
    df['value_source_value'] = df['value_source_value'].str.replace(r'(\d+)(\s+)(\++)', lambda x: x.groups()[2])
    df['value_source_value'] = df['value_source_value'].str.replace(r'(\++)(\d+)(\s+)', lambda x: x.groups()[0])
    
    # #replace + to Rh+ only for 'HJ1UHH1'(Rh type laboratory code) 
    # _rh_pos_df = df[df['measurement_concept_id'] == 'BL7002']['value_source_value'].str.replace('+','rh+')
    # _rh_neg_df = df[df['measurement_concept_id'] == 'BL7002']['value_source_value'].str.replace('-','rh-')
    # df.loc[df.measurement_concept_id == 'HJ1UHH1', 'value_source_value'] = _rh_pos_df 
    # df.loc[df.measurement_concept_id == 'HJ1UHH1', 'value_source_value'] = _rh_neg_df
    return df

df_with_freq = df.groupby(df.columns.tolist()).size().reset_index(name = 'Freq')
df_with_freq_sort = df_with_freq.sort_values(by = 'Freq',ascending = False)
df_cleaned = _general_preprocessing(df_with_freq_sort)
df_cleaned_regrouped = df_cleaned.groupby(['measurement_concept_id','value_source_value'])
df_cleaned_sort = df_cleaned_regrouped.sum().reset_index().sort_values(by = 'Freq', ascending = False)

dist_dicts = defaultdict(dict)

for cid, value, freq in df_cleaned_sort.values:
    dist_dicts[cid][value] = freq
    
dist_dicts = dict(dist_dicts)

/tmp/ipykernel_1267/1898590534.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['value_source_value'] = df['value_source_value'].str.replace('(',"")
/tmp/ipykernel_1267/1898590534.py:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['value_source_value'] = df['value_source_value'].str.replace(')',"")
/tmp/ipykernel_1267/1898590534.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df['value_source_value'] = df['value_source_value'].str.replace(r'(\d)(\+)', lambda x: x.groups()[1]*int(x.groups()[0]))
/tmp/ipykernel_1267/1898590534.py:17: FutureWarning: The default value of regex will change from True to False in a f

In [23]:
# 2) Laboratory tests Categorization
import re
import collections, re
import json


def get_majors_and_minors(dist_dict, acc_thr=100, diff_thr=100, n_thr=9):

    if len(dist_dict) < 5:
        return dist_dict

    total = sum(dist_dict.values())
    
    tmp = {}
    sorted_dist_dict = sorted(dist_dict.items(), key=lambda t: -t[1])
    
    for idx, (k, v) in enumerate(sorted_dist_dict):
        
        if idx==0:
            prev = v
            prev_acc = 0

        acc_ratio = (prev_acc + v) / total * 100
        
        #Escape Rules
        if (acc_ratio >= acc_thr): 
            tmp.update({k:v})
            break
            
        if (idx>n_thr): 
            break
            
        tmp.update({k:v})
        prev = v
        prev_acc += v
        
    return tmp


def get_groups(majors,
    rules_A = ['straw','clear','cloudy','yellow','brown','orange','amber','Deep Straw'],
    rules_B = ['compatible','incompatible', 'compitable', 'comaptible'],
    rules_C = ['positive', 'negative'],
    rules_D = ['ab','a','b','o'],
    rules_E = ['reative','non-reactive','non reactive'],
    rules_F = ['normal','absent', 'few','+']
    # rules_D = ['ab','rh+'],
    # rules_E = ['rea.*', 'non.*']):
              ):

    group_A = [] 
    group_B = [] 
    group_C = [] 
    group_D = []  
    group_E = []
    group_F = []

    # message_form = '\n'.join(['Rules_%s: {}' % c for c in 'ABCDE'])
    # print(message_form.format(rules_A, rules_B, rules_C,rules_D, rules_E))
    
    def is_group_A(major_dict):
        return sum([1 for k in major_dict.keys() if k in rules_A])
    def is_group_B(major_dict):
        return sum([1 for k in major_dict.keys() if k in rules_B])
    def is_group_C(major_dict):
        return sum([1 for s in major_dict.keys() for r in rules_C])
    def is_group_D(major_dict):
        return sum([1 for k in major_dict.keys() if k in rules_D])
    def is_group_E(major_dict):
        return sum([1 for s in major_dict.keys() for r in rules_E])
    def is_group_F(major_dict):
        return sum([1 for s in major_dict.keys() for r in rules_F ])

    for table_cid, major_dict in majors.items():
        
        if is_group_A(major_dict): 
            group_A.append(table_cid)
            
        elif is_group_B(major_dict):
            group_B.append(table_cid)
            
        elif is_group_C(major_dict):
            group_C.append(table_cid)
            
        elif is_group_D(major_dict): 
            group_D.append(table_cid)
            
        elif is_group_E(major_dict):
            group_E.append(table_cid)
        elif is_group_F(major_dict):
            group_F.append(table_cid)
            

    # merge_group_set = set(group_A + group_B + group_C + group_D + group_E)
    merge_group_set= set(group_A+ group_B+ group_C+ group_D + group_E + group_F)
    group_others = [table_cid for table_cid in majors.keys() if table_cid not in merge_group_set]
    return [group_A, group_B, group_C, group_D, group_E, group_F]
    # return [group_A, group_B, group_C, group_D, group_E, group_others]


def get_group_dict(group, regex=lambda x:x):
    
    group_dict = defaultdict(int)
               
    for table_cid in group:
        for k, v in dist_dicts[table_cid].items():
            group_dict[regex(re.sub(r'\s+', ' ', k))] += v
    
    return group_dict


majors = {table_cid:get_majors_and_minors(dist_dict, acc_thr=99.5) 
          for table_cid, dist_dict in dist_dicts.items()}

# print("majors \n")
# print(majors)
groups = get_groups(majors)


groups_dict = [get_group_dict(group) for group in groups] 

# print("groups_dict \n")
# print(groups_dict)
groups_sample = [group_dict.keys() for group_dict in groups_dict]

# print("groups_sample \n")
# print(groups_sample)

In [28]:
group = [['Moderately Positive',
 'Moderately positive',
 'Nagetive',
 'Negative',
 'Negative.',
 'Negetive',
 'Non Reactive',
 'Non-Reactive',
 'Positive',
 'Postive',
 'Reactive',
 'Weakly Positive',
 'Weakly positive',
 '\\Negative'], ['+',
 '++',
 '+++',
 '++++',
 '+',
 '++',
 '+++',
 '++++',
 'A Few',
 'A few',
 'Absent',
 'Few',
 'N/S',
 'N/s',
 'NIl',
 'Nil',
 'No microorganism seen.',
 'Normal',
 'Normal.',
 'Not Done',
 'Occasional',
 'Plenty',
 'Present',
 'Present (+)',
 'Present (++)',
 'Trace',
 'Very few',
 'a few',
 'nil']
]   

In [54]:
from itertools import chain
import pandas as pd
from scipy import spatial


class Mapper():
    def __init__(self, samples, ref_list):
        self.samples = samples
        self.ref_list = ref_list
        
    def create_feature_matrix(self, samples1, samples2):
        def _create_feature_matrix(samples, refs):
            #refs는 feature list
            result = []
            row_idx = []
            for r_idx, ng_str in enumerate(samples):
                result.append([0]*len(refs))
                for ng_char in ng_str:
                    result[r_idx][refs.index(ng_char)] += 1
            return result

        samples1 = [s.lower() for s in samples1]
        samples2 = [s.lower() for s in samples2]
        refs = list(set(chain.from_iterable(samples1+samples2)))
        matrix1 = _create_feature_matrix(samples1, refs)
        matrix2 = _create_feature_matrix(samples2, refs)
        return matrix1, matrix2
    
    def assign_to_ref(self, sample_vectors, ref_vectors, sample_list, ref_list):
        def cosine_similarity_matrix(matrix1, matrix2):

            distance = []
            for s_idx, sample in enumerate(matrix1):
                distance.append([])
                for ref in matrix2:
                    distance[s_idx].append(
                        round(1-spatial.distance.cosine(sample, ref),2)
                    )
            return distance
        
        def lp_dist_matrix(matrix1, matrix2):
            distance = []
            for s_idx, sample in enumerate(matrix1):
                distance.append([])
                for ref in matrix2:
                    distance[s_idx].append(
                        round(spatial.distance.euclidean(sample, ref),2)
                    )
            return distance
        
        def levenshtein_measure(sample_list, ref_list):
            def levenshtein(s1, s2):
            # based on Wikipedia/Levenshtein_distance
                if len(s1) < len(s2):
                    return levenshtein(s2, s1)

                if len(s2) == 0:
                    return len(s1)
    
                previous_row = range(len(s2) + 1)
                for i, c1 in enumerate(s1):
                    current_row = [i + 1]
                    for j, c2 in enumerate(s2):
                        insertions = previous_row[j + 1] + 1
                        deletions = current_row[j] + 1
                        substitutions = previous_row[j] + (c1 != c2)
                        current_row.append(min(insertions, deletions, substitutions))
                    previous_row = current_row
    
                return previous_row[-1]
    
            distance = []
            for s_idx, sample in enumerate(sample_list):
                distance.append([])
                for ref in ref_list:
                    distance[s_idx].append(levenshtein(sample, ref))
            return distance
        
        
        # Python3 implementation of above approach
        from math import floor
        def jaro_winkler_measures(sample_list, ref_list):
            # Function to calculate the
            # Jaro Similarity of two strings
            def jaro_distance(s1, s2) :

                # If the strings are equal
                if (s1 == s2) :
                    return 1.0;

                # Length of two strings
                len1 = len(s1);
                len2 = len(s2);

                if (len1 == 0 or len2 == 0) :
                    return 0.0;

                # Maximum distance upto which matching
                # is allowed
                max_dist = (max(len(s1), len(s2)) // 2 ) - 1;
                
                 # Count of matches
                match = 0;

                # Hash for matches
                hash_s1 = [0] * len(s1) ;
                hash_s2 = [0] * len(s2) ;

                # Traverse through the first string
                for i in range(len1) :

                    # Check if there is any matches
                    for j in range( max(0, i - max_dist),
                                min(len2, i + max_dist + 1)) :

                        # If there is a match
                        if (s1[i] == s2[j] and hash_s2[j] == 0) :
                            hash_s1[i] = 1;
                            hash_s2[j] = 1;
                            match += 1;
                            break;

                # If there is no match
                if (match == 0) :
                    return 0.0;
                # Number of transpositions
                t = 0;

                point = 0;

                # Count number of occurrences
                # where two characters match but
                # there is a third matched character
                # in between the indices
                for i in range(len1) :
                    if (hash_s1[i]) :

                        # Find the next matched character
                        # in second string
                        while (hash_s2[point] == 0) :
                            point += 1;

                        if (s1[i] != s2[point]) :
                            point += 1;
                            t += 1;
                        else :
                            point += 1;
                    t /= 2;

                # Return the Jaro Similarity
                return ((match / len1 + match / len2 +
                        (match - t) / match ) / 3.0);

            # Jaro Winkler Similarity
            def jaro_Winkler(s1, s2) :

                jaro_dist = jaro_distance(s1, s2);

                # If the jaro Similarity is above a threshold
                if (jaro_dist > 0.7) :

                    # Find the length of common prefix
                    prefix = 0;

                    for i in range(min(len(s1), len(s2))) :

                        # If the characters match
                        if (s1[i] == s2[i]) :
                            prefix += 1;
                        # Else break
                        else :
                            break;

                    # Maximum of 4 characters are allowed in prefix
                    prefix = min(4, prefix);

                    # Calculate jaro winkler Similarity
                    jaro_dist += 0.1 * prefix * (1 - jaro_dist);
                
                return jaro_dist;
            distance = []
            for s_idx, sample in enumerate(sample_list):
                distance.append([])
                for ref in ref_list:
                    distance[s_idx].append(jaro_Winkler(sample, ref))
            return distance
        
            
           




        sample_list = list(sample_list)

        cos_scores = cosine_similarity_matrix(sample_vectors, ref_vectors)
        lev_scores  = levenshtein_measure(sample_list, ref_list)
        lp_dist_scores = lp_dist_matrix(sample_vectors, ref_vectors)
        jw_scores = jaro_winkler_measures(sample_list, ref_list)
        
        result = []
        outlier = []
        for s_idx, sample in enumerate(sample_vectors):
            max_cos_score = max(cos_scores[s_idx])
            max_cos_idx = cos_scores[s_idx].index(max_cos_score)
            max_cos_cnt = cos_scores[s_idx].count(max_cos_score)
            min_lp_dist_score = min(lp_dist_scores[s_idx])
            min_lp_dist_idx = lp_dist_scores[s_idx].index(min_lp_dist_score)
            min_lp_dist_cnt = lp_dist_scores[s_idx].count(min_lp_dist_score)
            min_lev_score = min(lev_scores[s_idx])
            min_lev_idx = lev_scores[s_idx].index(min_lev_score)
            max_jw_score = max(jw_scores[s_idx])
            max_jw_idx = jw_scores[s_idx].index(max_jw_score)
            max_jw_cnt = jw_scores[s_idx].count(max_jw_score)
            
            
            if max_cos_cnt == len(cos_scores[s_idx]):
                outlier.append(
                    [sample_list[s_idx],
                     ref_list[max_cos_idx],
                     cos_scores[s_idx]]
                )
            else:
                result.append(
                    [sample_list[s_idx],
                     ref_list[max_cos_idx],
                     cos_scores[s_idx],
                     ref_list[max_jw_idx],
                     jw_scores[s_idx],
                     ref_list[min_lp_dist_idx],
                     lp_dist_scores[s_idx],
                     ref_list[min_lev_idx],
                     lev_scores[s_idx],
                     'cosine' if max_cos_cnt <= 1 else 'Eucl',
                     ref_list[max_cos_idx] if max_cos_cnt <= 1 else ref_list[min_lp_dist_idx]]
                )
        result_df = pd.DataFrame(result, columns=['smp','ref','cos','ref_jw','jw','ref_lp','lp','ref_lev','lv','ecul','refe'])
        # result_df.to_csv("result_df.csv")
        return result, outlier
    
    def get_report_df(self, results):

        report_df = pd.DataFrame(
            results,
            columns=[
                'Data',
                'cos_word',
                'cos_score',
                'jw_word',
                'jw_score',
                'euc_word',
                'euc_score',
                'lev_word',
                'lev_score',
                'Measure',
                'Final'
            ]
        )
        
        report_df = report_df[['Data','jw_word','jw_score','cos_word','cos_score','euc_word','euc_score','lev_word','lev_score','Measure','Final']]
        
        return report_df
    
    def analyze(self):
        self._sample_vectors, self._ref_vectors = self.create_feature_matrix(
            self.samples, self.ref_list)
        
        self._cluster_results, self._outlier = self.assign_to_ref(
            self._sample_vectors, self._ref_vectors, self.samples, self.ref_list)
        
        self.report_df= self.get_report_df(self._cluster_results)

        
ref_list_A = ['straw', 'amber', 'brown', 'green', 'yellow', 'orange', 'black','blue', 'red', 'other', 'clear', 'cloudy', 'hazy', 'turbid', 'bloody']
ref_list_B = ['neg', 'trace', '+', '++', '+++', '++++']
ref_list_C = ['posi', 'neg-','weak-pos']
ref_list_D = ['a','b','c','ab','cisab','rh+','rh-','partiald','weakd','variantd'] 
ref_list_E = ['reac', 'non-reac','weak-reac']
ref_list_F= ['Moderately Positive', 'Negative','Positive','Weakly Positive','Reactive','Non Reactive']
ref_list_G = ['+','++','+++','A Few','N\S','Nil','No Migro-organism Seen','Normal','Not Done','Occasianal','Plenty','Present','Absent','Trace']   

groups_agents = [
    Mapper(sample, ref_list) for sample, ref_list in zip(
        group,
        [ref_list_F, ref_list_G]
    )
]

for agent in groups_agents:
    agent.analyze()

In [170]:
from itertools import chain
import pandas as pd
from scipy import spatial


class Mapper():
    def __init__(self, samples, ref_list):
        self.samples = samples
        self.ref_list = ref_list
        
    def create_feature_matrix(self, samples1, samples2):
        def _create_feature_matrix(samples, refs):
            #refs는 feature list
            result = []
            row_idx = []
            for r_idx, ng_str in enumerate(samples):
                result.append([0]*len(refs))
                for ng_char in ng_str:
                    result[r_idx][refs.index(ng_char)] += 1
            return result

        samples1 = [s.lower() for s in samples1]
        samples2 = [s.lower() for s in samples2]
        refs = list(set(chain.from_iterable(samples1+samples2)))
        matrix1 = _create_feature_matrix(samples1, refs)
        matrix2 = _create_feature_matrix(samples2, refs)
        return matrix1, matrix2
    
    def assign_to_ref(self, sample_vectors, ref_vectors, sample_list, ref_list):
        def cosine_similarity_matrix(matrix1, matrix2):

            distance = []
            for s_idx, sample in enumerate(matrix1):
                distance.append([])
                for ref in matrix2:
                    distance[s_idx].append(
                        round(1-spatial.distance.cosine(sample, ref),2)
                    )
            return distance
        
        def lp_dist_matrix(matrix1, matrix2):
            distance = []
            for s_idx, sample in enumerate(matrix1):
                distance.append([])
                for ref in matrix2:
                    distance[s_idx].append(
                        round(spatial.distance.euclidean(sample, ref),2)
                    )
            return distance
        
        def levenshtein_measure(sample_list, ref_list):
            def levenshtein(s1, s2):
            # based on Wikipedia/Levenshtein_distance
                if len(s1) < len(s2):
                    return levenshtein(s2, s1)

                if len(s2) == 0:
                    return len(s1)
    
                previous_row = range(len(s2) + 1)
                for i, c1 in enumerate(s1):
                    current_row = [i + 1]
                    for j, c2 in enumerate(s2):
                        insertions = previous_row[j + 1] + 1
                        deletions = current_row[j] + 1
                        substitutions = previous_row[j] + (c1 != c2)
                        current_row.append(min(insertions, deletions, substitutions))
                    previous_row = current_row
    
                return previous_row[-1]
    
            distance = []
            for s_idx, sample in enumerate(sample_list):
                distance.append([])
                for ref in ref_list:
                    distance[s_idx].append(levenshtein(sample, ref))
            return distance
        # Python3 implementation of above approach
        from math import floor
        def jaro_winkler_measures(sample_list, ref_list):
            # Function to calculate the
            # Jaro Similarity of two strings
            def jaro_distance(s1, s2) :

                # If the strings are equal
                if (s1 == s2) :
                    return 1.0;

                # Length of two strings
                len1 = len(s1);
                len2 = len(s2);

                if (len1 == 0 or len2 == 0) :
                    return 0.0;

                # Maximum distance upto which matching
                # is allowed
                max_dist = (max(len(s1), len(s2)) // 2 ) - 1;

                # Count of matches
                match = 0;

                # Hash for matches
                hash_s1 = [0] * len(s1) ;
                hash_s2 = [0] * len(s2) ;

                # Traverse through the first string
                for i in range(len1) :

                    # Check if there is any matches
                    for j in range( max(0, i - max_dist),
                                min(len2, i + max_dist + 1)) :

                        # If there is a match
                        if (s1[i] == s2[j] and hash_s2[j] == 0) :
                            hash_s1[i] = 1;
                            hash_s2[j] = 1;
                            match += 1;
                            break;

                # If there is no match
                if (match == 0) :
                    return 0.0;

                # Number of transpositions
                t = 0;

                point = 0;

                # Count number of occurrences
                # where two characters match but
                # there is a third matched character
                # in between the indices
                for i in range(len1) :
                    if (hash_s1[i]) :

                        # Find the next matched character
                        # in second string
                        while (hash_s2[point] == 0) :
                            point += 1;

                        if (s1[i] != s2[point]) :
                            point += 1;
                            t += 1;
                        else :
                            point += 1;

                    t /= 2;

                # Return the Jaro Similarity
                return ((match / len1 + match / len2 +
                        (match - t) / match ) / 3.0);

            # Jaro Winkler Similarity
            def jaro_Winkler(s1, s2) :

                jaro_dist = jaro_distance(s1, s2);

                # If the jaro Similarity is above a threshold
                if (jaro_dist > 0.7) :

                    # Find the length of common prefix
                    prefix = 0;

                    for i in range(min(len(s1), len(s2))) :

                        # If the characters match
                        if (s1[i] == s2[i]) :
                            prefix += 1;

                        # Else break
                        else :
                            break;

                    # Maximum of 4 characters are allowed in prefix
                    prefix = min(4, prefix);

                    # Calculate jaro winkler Similarity
                    jaro_dist += 0.1 * prefix * (1 - jaro_dist);

                return jaro_dist;
            
            distance = []
            for s_idx, sample in enumerate(sample_list):
                distance.append([])
                for ref in ref_list:
                    distance[s_idx].append(jaro_Winkler(sample, ref))
            return distance



        
        sample_list = list(sample_list)

        cos_scores = cosine_similarity_matrix(sample_vectors, ref_vectors)
        lev_scores  = levenshtein_measure(sample_list, ref_list)
        lp_dist_scores = lp_dist_matrix(sample_vectors, ref_vectors)
        jw_scores  = jaro_winkler_measures(sample_list, ref_list)
        
        result = []
        outlier = []
        for s_idx, sample in enumerate(sample_vectors):
            max_cos_score = max(cos_scores[s_idx])
            max_cos_idx = cos_scores[s_idx].index(max_cos_score)
            max_cos_cnt = cos_scores[s_idx].count(max_cos_score)
            max_jw_score = max(jw_scores[s_idx])
            max_jw_idx = jw_scores[s_idx].index(max_jw_score)
            max_jw_cnt = jw_scores[s_idx].count(max_jw_score )
            min_lp_dist_score = min(lp_dist_scores[s_idx])
            min_lp_dist_idx = lp_dist_scores[s_idx].index(min_lp_dist_score)
            min_lp_dist_cnt = lp_dist_scores[s_idx].count(min_lp_dist_score)
            min_lev_score = min(lev_scores[s_idx])
            min_lev_idx = lev_scores[s_idx].index(min_lev_score)
            
            if max_cos_cnt == len(cos_scores[s_idx]):
                outlier.append(
                    [sample_list[s_idx],
                     ref_list[max_cos_idx],
                     cos_scores[s_idx]]
                )
            else:
                result.append(
                    [sample_list[s_idx],
                     ref_list[max_cos_idx],
                     cos_scores[s_idx],
                     ref_list[min_lp_dist_idx],
                     lp_dist_scores[s_idx],
                     ref_list[min_lev_idx],
                     lev_scores[s_idx],
                     'cosine' if max_cos_cnt <= 1 else 'Eucl',
                     ref_list[max_cos_idx] if max_cos_cnt <= 1 else ref_list[min_lp_dist_idx]]
                )

        return result, outlier
    
    def get_report_df(self, results):

        report_df = pd.DataFrame(
            results,
            columns=[
                'Data',
                'cos_word',
                'cos_score',
                'euc_word',
                'euc_score',
                'lev_word',
                'lev_score',
                'Measure',
                'Final'
            ]
        )
        report_df.to_csv("r.csv")
        report_df = report_df[['Data','Final']]
        
        return report_df
    
    def analyze(self):
        self._sample_vectors, self._ref_vectors = self.create_feature_matrix(
            self.samples, self.ref_list)
        
        self._cluster_results, self._outlier = self.assign_to_ref(
            self._sample_vectors, self._ref_vectors, self.samples, self.ref_list)
        
        self.report_df= self.get_report_df(self._cluster_results)

        
ref_list_A = ['straw', 'amber', 'brown', 'green', 'yellow', 'orange', 'black','blue', 'red', 'other', 'clear', 'cloudy', 'hazy', 'turbid', 'bloody']
ref_list_B = ['neg', 'trace', '+', '++', '+++', '++++']
ref_list_C = ['posi', 'neg-','weak-pos']
ref_list_D = ['a','b','c','ab','cisab','rh+','rh-','partiald','weakd','variantd'] 
ref_list_E = ['reac', 'non-reac','weak-reac']

groups_agents = [
    Mapper(sample, ref_list) for sample, ref_list in zip(
        groups_sample[:-1],
        [ref_list_A, ref_list_B, ref_list_C,ref_list_D,ref_list_E]
    )
]

for agent in groups_agents:
    agent.analyze()

In [55]:
# 4) Result table
groups_agents[0].report_df

,Data,jw_word,jw_score,cos_word,cos_score,euc_word,euc_score,lev_word,lev_score,Measure,Final
0,Moderately Positive,Moderately Positive,"[1.0, 0.5701754385964912, 0.5109496534916392, ...",Moderately Positive,"[1, 0.68, 0.8, 0.87, 0.74, 0.72]",Moderately Positive,"[0.0, 4.12, 3.61, 2.83, 3.87, 3.87]",Moderately Positive,"[0, 13, 11, 6, 13, 13]",cosine,Moderately Positive
1,Moderately positive,Moderately Positive,"[0.9789473684210526, 0.5701754385964912, 0.510...",Moderately Positive,"[1, 0.68, 0.8, 0.87, 0.74, 0.72]",Moderately Positive,"[0.0, 4.12, 3.61, 2.83, 3.87, 3.87]",Moderately Positive,"[1, 13, 12, 7, 13, 13]",cosine,Moderately Positive
2,Nagetive,Negative,"[0.5669202302631579, 0.99853515625, 0.65104166...",Negative,"[0.68, 1, 0.6, 0.65, 0.8, 0.79]",Negative,"[4.12, 0.0, 2.83, 3.32, 2.0, 2.45]",Negative,"[13, 2, 4, 10, 4, 6]",cosine,Negative
3,Negative,Negative,"[0.5701754385964912, 1.0, 0.6510416666666666, ...",Negative,"[0.68, 1, 0.6, 0.65, 0.8, 0.79]",Negative,"[4.12, 0.0, 2.83, 3.32, 2.0, 2.45]",Negative,"[13, 0, 4, 10, 3, 6]",cosine,Negative
4,Negative.,Negative,"[0.5516569200779727, 0.9777777777777777, 0.640...",Negative,"[0.65, 0.95, 0.57, 0.62, 0.76, 0.75]",Negative,"[4.24, 1.0, 3.0, 3.46, 2.24, 2.65]",Negative,"[14, 1, 5, 11, 4, 7]",cosine,Negative
5,Negetive,Negative,"[0.5005482456140351, 0.9416666666666667, 0.651...",Negative,"[0.67, 0.93, 0.59, 0.61, 0.76, 0.73]",Negative,"[4.12, 1.41, 3.16, 3.61, 2.45, 2.83]",Negative,"[13, 1, 4, 10, 3, 6]",cosine,Negative
6,Non Reactive,Non Reactive,"[0.6557859900402047, 0.8375000000000001, 0.650...",Non Reactive,"[0.72, 0.79, 0.55, 0.63, 0.79, 1]",Non Reactive,"[3.87, 2.45, 3.46, 3.61, 2.45, 0.0]",Non Reactive,"[13, 6, 7, 10, 4, 0]",cosine,Non Reactive
7,Non-Reactive,Non Reactive,"[0.6089181286549707, 0.8375000000000001, 0.650...",Non Reactive,"[0.67, 0.79, 0.55, 0.57, 0.79, 0.94]",Non Reactive,"[4.12, 2.45, 3.46, 3.87, 2.45, 1.41]",Non Reactive,"[13, 6, 7, 11, 4, 1]",cosine,Non Reactive
8,Positive,Positive,"[0.4484649122807018, 0.6588541666666666, 1.0, ...",Positive,"[0.8, 0.6, 1, 0.8, 0.6, 0.55]",Positive,"[3.61, 2.83, 0.0, 2.65, 2.83, 3.46]",Positive,"[11, 4, 0, 7, 4, 7]",cosine,Positive
9,Postive,Positive,"[0.4663220551378446, 0.6904761904761904, 0.964...",Positive,"[0.81, 0.6, 0.96, 0.78, 0.6, 0.57]",Positive,"[3.74, 2.65, 1.0, 2.83, 2.65, 3.32]",Positive,"[12, 4, 1, 8, 4, 7]",cosine,Positive


In [56]:
groups_agents[1].report_df

,Data,jw_word,jw_score,cos_word,cos_score,euc_word,euc_score,lev_word,lev_score,Measure,Final
0,+,+,"[1.0, 0.8500000000000001, 0.7999999999999999, ...",+,"[1, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",+,"[0.0, 1.0, 2.0, 2.45, 2.0, 2.0, 7.0, 2.65, 3.6...",+,"[0, 1, 2, 5, 3, 3, 22, 6, 8, 10, 6, 7, 6, 5]",Eucl,+
1,++,++,"[0.8500000000000001, 1.0, 0.9111111111111111, ...",+,"[1, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",++,"[1.0, 0.0, 1.0, 3.0, 2.65, 2.65, 7.21, 3.16, 4...",++,"[1, 0, 1, 5, 3, 3, 22, 6, 8, 10, 6, 7, 6, 5]",Eucl,++
2,+++,+++,"[0.7999999999999999, 0.9111111111111111, 1.0, ...",+,"[1, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",+++,"[2.0, 1.0, 0.0, 3.74, 3.46, 3.46, 7.55, 3.87, ...",+++,"[2, 1, 0, 5, 3, 3, 22, 6, 8, 10, 6, 7, 6, 5]",Eucl,+++
3,++++,+++,"[0.775, 0.8666666666666667, 0.9416666666666667...",+,"[1, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",+++,"[3.0, 2.0, 1.0, 4.58, 4.36, 4.36, 8.0, 4.69, 5...",+++,"[3, 2, 1, 5, 4, 4, 22, 6, 8, 10, 6, 7, 6, 5]",Eucl,+++
4,+,+,"[1.0, 0.8500000000000001, 0.7999999999999999, ...",+,"[1, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",+,"[0.0, 1.0, 2.0, 2.45, 2.0, 2.0, 7.0, 2.65, 3.6...",+,"[0, 1, 2, 5, 3, 3, 22, 6, 8, 10, 6, 7, 6, 5]",Eucl,+
5,++,++,"[0.8500000000000001, 1.0, 0.9111111111111111, ...",+,"[1, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",++,"[1.0, 0.0, 1.0, 3.0, 2.65, 2.65, 7.21, 3.16, 4...",++,"[1, 0, 1, 5, 3, 3, 22, 6, 8, 10, 6, 7, 6, 5]",Eucl,++
6,+++,+++,"[0.7999999999999999, 0.9111111111111111, 1.0, ...",+,"[1, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",+++,"[2.0, 1.0, 0.0, 3.74, 3.46, 3.46, 7.55, 3.87, ...",+++,"[2, 1, 0, 5, 3, 3, 22, 6, 8, 10, 6, 7, 6, 5]",Eucl,+++
7,++++,+++,"[0.775, 0.8666666666666667, 0.9416666666666667...",+,"[1, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",+++,"[3.0, 2.0, 1.0, 4.58, 4.36, 4.36, 8.0, 4.69, 5...",+++,"[3, 2, 1, 5, 4, 4, 22, 6, 8, 10, 6, 7, 6, 5]",Eucl,+++
8,A Few,A Few,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.4151515151515...",A Few,"[0.0, 0.0, 0.0, 1, 0.0, 0.0, 0.32, 0.18, 0.26,...",A Few,"[2.45, 3.0, 3.74, 0.0, 2.83, 2.83, 6.56, 3.0, ...",A Few,"[5, 5, 5, 0, 5, 5, 20, 6, 7, 10, 6, 6, 4, 5]",cosine,A Few
9,A few,A Few,"[0.0, 0.0, 0.0, 0.8933333333333333, 0.0, 0.0, ...",A Few,"[0.0, 0.0, 0.0, 1, 0.0, 0.0, 0.32, 0.18, 0.26,...",A Few,"[2.45, 3.0, 3.74, 0.0, 2.83, 2.83, 6.56, 3.0, ...",A Few,"[5, 5, 5, 1, 5, 5, 20, 6, 7, 10, 6, 6, 4, 5]",cosine,A Few


In [8]:
groups_agents[2].report_df

,Data,Final
0,neg -,neg-
1,pos +,posi
2,posi +,posi
3,negative,neg-
4,pos(11.00),posi
...,...,...
2031,pos(4),posi
2032,pos(3),posi
2033,pos,posi
2034,positive (>300.0),posi


In [9]:
groups_agents[3].report_df

,Data,Final
0,a,a
1,b,b
2,ab,ab
3,bmt,b
4,a2b,ab
5,a2b3,ab
6,a1b3,ab
7,a3,a
8,b3,b


In [10]:
groups_agents[4].report_df

,Data,Final
0,non-reactive(0.01),non-reac
1,non-reactive(0.02),non-reac
2,non-reactive(0.03),non-reac
3,non-reactive(0.05),non-reac
4,non-reactive(0.04),non-reac
...,...,...
170,w-rea(1:1),weak-reac
171,react(1:1),reac
172,non-reactive(csf),non-reac
173,weakly-reactive(1:1),weak-reac
